# EDA: postprocessing and connectivity with tableau dashboard

File: 'core_classroom_analysis.py'

Checkpoint: 1

## Load data

In [13]:
from pprint import pprint
import pandas as pd
import numpy as np
import datetime

from dask import dataframe as dd
from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

import geoip2.database

In [14]:
# Load checkpoint shelve
import pickle

pkl_filepath = '/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/core_classroom_analysis_cp1.pkl'
with open(pkl_filepath, 'rb') as f:
    inparams = pickle.load(f)

user_activity_blocks_df = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_user_activity_blocks_df.pkl')
jos_users = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_jos_users.pkl')
toolrun_df = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_toolrun_df.pkl')
cluster_output_candidate = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_cluster_output_candidate.pkl')
detected_clusters_df = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_detected_clusters_df.pkl')

cluster_post_sychrony = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_cluster_post_sychrony.pkl')
students_info_df = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_students_info_df.pkl')
class_info_df = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_class_info_df.pkl')
classtool_info_df = pd.read_pickle('/home/wang159/nanoHUB/projects/online_users_ts_analysis/temp/cp1_classtool_info_df.pkl')

In [15]:
class_info_df

,class_id,start,end,lon,lat,size,ip_set
0,0,2018-01-15,2018-01-21,-86.918439,40.431720,17,"[{68.234.144.173, kkurpius, 128.211.186.10, rm..."
1,1,2018-02-05,2018-04-19,-86.916783,40.430048,45,"[{98.223.101.154, 192.31.0.5, sheng18, su107, ..."
2,2,2018-02-26,2018-03-02,-86.918942,40.432611,6,"[{dbradbu, dtisdale, owellbor, 128.211.218.138..."
3,3,2018-04-11,2018-04-12,-86.919060,40.432820,5,"[{128.46.217.23, mason92, rleather, hong224, 1..."
4,4,2018-04-12,2018-04-13,-86.919060,40.432820,5,"[{kline40, owellbor, 128.211.218.138, 68.234.1..."
...,...,...,...,...,...,...,...
142,142,2018-02-14,2018-02-14,-83.023500,39.965300,6,"[{18.189.46.59, 18.189.36.34, 18.189.16.157, 1..."
143,143,2018-03-26,2018-03-27,-86.917215,40.429554,7,"[{72.12.194.66, robin295, cortez10, 128.211.24..."
144,144,2018-02-20,2018-04-26,-1.485918,53.364603,44,"[{elt17bw, gary0010, nozdormu0129, 143.167.235..."
145,145,2018-02-06,2018-02-10,-86.908056,40.404208,17,"[{128.211.249.89, kcruse2, 199.66.69.246, jmst..."


# City information

In [20]:
# TEST
IP_list = ['128.46.78.247', '128.46.78.243', '128.46.78.242']

ip_df = pd.DataFrame(IP_list, columns=['ip'])

In [54]:
def get_city_info(ip):
    
    reader = geoip2.database.Reader(inparams[0].geoip2_mmdb_filepath)
    response = reader.city(ip)
    
    try:
        city_name = response.city.names['en']
    except:
        city_name = ''
        
    try:
        state_name = response.subdivisions[0].names['en']
    except:
        state_name = ''
    
    
    try:
        country_name = response.country.names['en']
    except:
        country_name = ''
    
    try:
        continent_name = response.continent.code
    except:
        continent_name = ''
    
    address_list = [city_name, state_name, country_name, continent_name]
    
    address = ', '.join(list(filter(None, address_list)))
    
    return city_name, state_name, ', '.join(list(filter(None, [country_name, continent_name])))

In [59]:
ip_df['city'], ip_df['state'], ip_df['country'] = zip(*ip_df.ip.apply(get_city_info))
display(ip_df)


# vote to identify the identity of the location
overall_city = ip_df.city.mode()
overall_state = ip_df.state.mode()
overall_country = ip_df.country.mode()

,ip,city,state,country
0,128.46.78.247,West Lafayette,Indiana,"United States, NA"
1,128.46.78.243,West Lafayette,Indiana,"United States, NA"
2,128.46.78.242,West Lafayette,Indiana,"United States, NA"


# School information within the US